# Light Curve Analysis Exploration

This notebook demonstrates the complete workflow for processing, visualizing, and extracting features from light curves.

## Setup

In [ ]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import our modules
from data_loader import load_npz_curve, preprocess_lightcurve
from visualization import (plot_lightcurve, plot_folded_curve, 
                          plot_feature_distribution, plot_comprehensive_analysis)
from feature_extraction import extract_features
from feature_pruning import manual_prune, analyze_feature_importance

# Set up plotting
plt.style.use('seaborn-v0_8')
%matplotlib inline

## 1. Create Synthetic Light Curve Data

For demonstration, we'll create a synthetic light curve with transit signals.

In [ ]:
# Create synthetic light curve with transits
np.random.seed(42)

# Parameters
n_points = 2000
time_span = 100  # days
period = 8.5  # days
transit_depth = 0.015
transit_duration = 0.3  # days
noise_level = 0.003

# Generate time series
time = np.sort(np.random.uniform(0, time_span, n_points))
flux = np.ones_like(time)

# Add transit signals
phase = (time / period) % 1.0
transit_phase = 0.2  # Transit occurs at phase 0.2

# Create transit shape (simplified box model)
for i, ph in enumerate(phase):
    phase_diff = min(abs(ph - transit_phase), abs(ph - transit_phase + 1), abs(ph - transit_phase - 1))
    if phase_diff < (transit_duration / period / 2):
        # Simple box transit
        flux[i] -= transit_depth

# Add stellar variability (sinusoidal)
stellar_period = 25.0  # rotation period
stellar_amplitude = 0.008
flux += stellar_amplitude * np.sin(2 * np.pi * time / stellar_period)

# Add noise
flux += np.random.normal(0, noise_level, len(flux))
flux_err = np.full_like(flux, noise_level)

print(f"Created synthetic light curve:")
print(f"  Time span: {time_span} days")
print(f"  Data points: {n_points}")
print(f"  Transit period: {period} days")
print(f"  Transit depth: {transit_depth*100:.1f}%")
print(f"  Noise level: {noise_level*100:.1f}%")

## 2. Load and Preprocess Data

In a real scenario, you would load from an .npz file. Here we'll use our synthetic data.

In [ ]:
# Create light curve data structure
lc_data = {
    'time': time,
    'flux': flux,
    'flux_err': flux_err
}

print("Raw light curve statistics:")
print(f"  Mean flux: {np.mean(flux):.6f}")
print(f"  Flux std: {np.std(flux):.6f}")
print(f"  Flux range: {np.max(flux) - np.min(flux):.6f}")

# Apply preprocessing
print("\nApplying preprocessing...")
processed_lc = preprocess_lightcurve(lc_data, apply_preprocessing=True)

print("\nProcessed light curve statistics:")
print(f"  Mean flux: {np.mean(processed_lc['flux']):.6f}")
print(f"  Flux std: {np.std(processed_lc['flux']):.6f}")
print(f"  Data points: {len(processed_lc['time'])}")

## 3. Visualize Light Curve

In [ ]:
# Plot raw light curve
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Raw data
ax1.scatter(lc_data['time'], lc_data['flux'], alpha=0.6, s=8, label='Raw data')
ax1.set_xlabel('Time (days)')
ax1.set_ylabel('Flux')
ax1.set_title('Raw Light Curve')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Processed data
ax2.scatter(processed_lc['time'], processed_lc['flux'], alpha=0.6, s=8, 
           color='orange', label='Processed data')
ax2.set_xlabel('Time (days)')
ax2.set_ylabel('Normalized Flux')
ax2.set_title('Processed Light Curve')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Phase-Folded Light Curve

In [ ]:
# Plot folded light curve using known period
fig, axes = plot_folded_curve(
    processed_lc['time'], 
    processed_lc['flux'], 
    period,
    flux_err=processed_lc['flux_err'],
    title=f"Phase-Folded Light Curve (P={period} days)"
)

plt.show()

## 5. Feature Extraction

Extract comprehensive features from the light curve.

In [ ]:
# Extract features
print("Extracting features...")
features_df = extract_features(
    processed_lc['time'], 
    processed_lc['flux'], 
    processed_lc['flux_err']
)

print(f"\nExtracted {len(features_df.columns)} features")
print(f"Feature categories:")

# Group features by category
categories = {
    'Basic Stats': [c for c in features_df.columns if any(x in c for x in ['mean', 'std', 'var', 'median'])],
    'Time Domain': [c for c in features_df.columns if any(x in c for x in ['autocorr', 'diff', 'trend'])],
    'Frequency': [c for c in features_df.columns if any(x in c for x in ['spectral', 'ls_', 'freq'])],
    'Variability': [c for c in features_df.columns if any(x in c for x in ['amplitude', 'stetson', 'rms'])],
    'Transit': [c for c in features_df.columns if any(x in c for x in ['transit', 'dip'])],
}

for cat, features in categories.items():
    print(f"  {cat}: {len(features)} features")

## 6. Examine Key Features

In [ ]:
# Display some key features
key_features = [
    'mean', 'std', 'amplitude', 'skewness', 'kurtosis',
    'ls_peak_power', 'ls_peak_period', 'spectral_entropy',
    'stetson_j', 'von_neumann_ratio',
    'deepest_dip_depth', 'n_dips'
]

print("Key Feature Values:")
print("=" * 50)
for feature in key_features:
    if feature in features_df.columns:
        value = features_df[feature].iloc[0]
        if isinstance(value, float) and not np.isnan(value):
            print(f"{feature:25s}: {value:10.6f}")
        else:
            print(f"{feature:25s}: {str(value):>10s}")
    else:
        print(f"{feature:25s}: {'Not found':>10s}")

# Check if we detected the period correctly
if 'ls_peak_period' in features_df.columns:
    detected_period = features_df['ls_peak_period'].iloc[0]
    if not np.isnan(detected_period):
        print(f"\nPeriod Detection:")
        print(f"  True period: {period:.2f} days")
        print(f"  Detected period: {detected_period:.2f} days")
        print(f"  Error: {abs(detected_period - period):.2f} days ({abs(detected_period - period)/period*100:.1f}%)")

## 7. Feature Analysis and Visualization

In [ ]:
# Plot feature distributions
fig, axes = plot_feature_distribution(features_df, max_features=16)
plt.show()

In [ ]:
# Analyze feature importance (by variance)
importance = analyze_feature_importance(features_df, method='variance')

print("Top 20 Most Variable Features:")
print("=" * 50)
for i, (feature, var) in enumerate(importance.head(20).items(), 1):
    print(f"{i:2d}. {feature:30s}: {var:.4f}")

## 8. Feature Selection and Pruning

Select a subset of features for analysis.

In [ ]:
# Select interesting features manually
selected_features = [
    # Basic statistics
    'mean', 'std', 'amplitude', 'skewness', 'kurtosis',
    
    # Variability
    'stetson_j', 'von_neumann_ratio', 'welch_stetson',
    
    # Periodicity
    'ls_peak_power', 'ls_peak_period', 'autocorr_lag_1',
    
    # Frequency domain
    'spectral_entropy', 'spectral_centroid', 'dominant_freq',
    
    # Transit features
    'deepest_dip_depth', 'n_dips', 'transit_ingress_slope',
    
    # Shape features
    'asymmetry_ratio', 'frac_above_2sigma', 'max_run_above_mean'
]

# Filter to features that actually exist
selected_features = [f for f in selected_features if f in features_df.columns]

print(f"Selected {len(selected_features)} features for analysis:")
for feature in selected_features:
    print(f"  - {feature}")

# Create pruned dataset
pruned_features = manual_prune(features_df, selected_features)
print(f"\nPruned dataset shape: {pruned_features.shape}")

## 9. Save Results

In [ ]:
# Save results to CSV files
features_df.to_csv('../output/all_features.csv', index=False)
pruned_features.to_csv('../output/selected_features.csv', index=False)

print("Results saved:")
print(f"  All features: ../output/all_features.csv ({len(features_df.columns)} features)")
print(f"  Selected features: ../output/selected_features.csv ({len(selected_features)} features)")

# Also save the light curve data
lc_df = pd.DataFrame({
    'time': processed_lc['time'],
    'flux': processed_lc['flux'],
    'flux_err': processed_lc['flux_err']
})
lc_df.to_csv('../output/processed_lightcurve.csv', index=False)
print(f"  Light curve: ../output/processed_lightcurve.csv ({len(lc_df)} points)")

## 10. Summary

This notebook demonstrated the complete workflow:

1. **Data Creation**: Generated synthetic transit light curve
2. **Preprocessing**: Applied cleaning and normalization
3. **Visualization**: Created light curve and folded plots
4. **Feature Extraction**: Computed 100+ astronomical features
5. **Analysis**: Examined feature distributions and importance
6. **Pruning**: Selected relevant features for analysis
7. **Export**: Saved results to CSV files

### Next Steps

- Use this workflow with your own `.npz` light curve files
- Explore different feature selection strategies
- Add custom features specific to your science case
- Use the extracted features for machine learning classification or regression

### Key Features Detected

The analysis successfully detected:
- Transit signals in the light curve
- Correct period estimation (if Lomb-Scargle worked)
- Variability characteristics
- Statistical properties of the data

This demonstrates the power of comprehensive feature extraction for astronomical time series analysis!